# Introduction to Image Preprocessing

## What is image preprocessing and why is it important?

Image data preprocessing is the process of manipulating raw image data into a usable and meaningful format. It allows you to eliminate unwanted distortions and enhance specific qualities essential for computer vision applications. Preprocessing is a crucial first step to prepare your image data before feeding it into machine learning models or algorithms. Though, it is one of the most under-explored problems in the data science community. Every developer has a unique way of doing it... 

The images should have small size so that the number of features is not large enough while feeding the images into a neural network. For example, a coloured image is $600 \times 800$ large, then the neural network need to handle $600 \times 800 \times 3 = 1440000$ parameters, which is quite large. On the other hand, any coloured image of $64 \times 64$ size needs only $64 \times 64 \times 3 = 12288$ parameters, which is fairly low and will be computationally efficient. Moreover, there can be great benefit in preparing the image pixel values prior to modelling, such as simply scaling pixel values to the range $[0,1]$ to centering and even standardising the values.

In this tutorial, you’ll learn all the tips and tricks for preparing your images for analysis using Python. We’ll cover everything from resizing and cropping to reducing noise and normalizing. By the time you’re done, your images will be ready for their closeup. With the help of libraries like [scipy](https://scipy.org/) and [scikit-image](https://scikit-image.org/), you’ll be enhancing images in no time. So get ready to roll up your sleeves — it’s time to dive into the complete guide to image preprocessing techniques in Python!

There are several techniques used in image preprocessing:

* **Grayscaling:** Converting color images to grayscale can simplify your image data and reduce computational needs for some algorithms
* **Resizing:** Resizing images to a uniform size is important for machine learning algorithms to function properly
* **Normalization:** Normalization adjusts the intensity values of pixels to a desired range, often between $0$ to $1$. This can improve the performance of machine learning models.
* **Noise reduction:** Smoothing, blurring, and filtering techniques can be applied to remove unwanted noise from images
* **Binarization:** Binarization converts grayscale images to black and white by thresholding
* **Contrast enhancement:** The contrast of images can be adjusted using histogram transformations

With the right combination of these techniques, you can significantly improve your image data and build better computer vision applications. Image preprocessing allows you to refine raw images into a format suitable for the problem you want to solve.

## Getting started with image preprocessing in Python

In [ ]:
#importing required Libraries
import numpy as np
import os
import glob 
import random 
import matplotlib.pyplot as plt  
%matplotlib inline

import scipy


# scientific programming
import numpy as np           

# display   
import matplotlib.pyplot as plt   


# image processing
import skimage 
import skimage.io as io
import skimage.color as color
from skimage import data

## 1. Converting images to grayscale

Grayscale conversion is simply converting images from colored to black and white so as to collapse the RGB channels into a single grayscale channel. It is normally used to reduce computation complexity in machine learning algorithms. Indeed, since most pictures don’t need color to be recognized, it is wise to use grayscale, which **reduces the number of pixels in an image**, thus, reducing the computations required.


Before we start exploring preprocessing techniques, let’s first explore the RGB channels of our original image.

In [ ]:
#Plotting the original image and the RGB channels  

i, (im1, im2, im3, im4) = plt.subplots(1, 4)
i.set_figwidth(15) 

image = io.imread('./images/skin.jpg')

im1.imshow(image)  #Original image
im2.imshow(image[:, : , 0]) #Red
im3.imshow(image[:, : , 1]) #Green
im4.imshow(image[:, : , 2]) #Blue
i.suptitle('Original & RGB skin image channels',y=0.8)
plt.show()

What’s that weird grayscale image? It’s rather green… 😲

This actually is a grayscale image (the image has only one band). But the default colormap provided with `imshow` is a gradient of blue and green. To display it in "real" gray levels, you must specify the option `cmap` in `imshow`. A large number of colormaps are available (see [catalog](https://matplotlib.org/stable/gallery/color/colormap_reference.html)); here we choose the classic `gray` colormap:

**Note:** To set the gray colormap by default, you can use the following statement: <code>plt.rcParams["image.cmap"]="gray"</code>

In [ ]:
i, (im1, im2, im3, im4) = plt.subplots(1, 4)
i.set_figwidth(15) 
im1.imshow(image)  #Original image
im2.imshow(image[:, : , 0],cmap='gray') #Red
im3.imshow(image[:, : , 1],cmap='gray') #Green
im4.imshow(image[:, : , 2],cmap='gray') #Blue
i.suptitle('Original & RGB skin image channels',y=0.8)
plt.show()

Now, execute the code below to convert the original image to grayscale. 

This example converts an image with RGB channels into an image with a single grayscale channel. The value of each grayscale pixel is calculated as the weighted sum of the corresponding red, green and blue pixels as:

```python
Y = 0.2125 R + 0.7154 G + 0.0721 B
```
*These weights are used by CRT phosphors as they better represent human perception of red, green and blue than equal weights.*

In [ ]:
gray_image = skimage.color.rgb2gray(image)
plt.imshow(gray_image, cmap = 'gray')
plt.title('Skin image (grayscale)')
plt.colorbar();

The grayscale image has intensities in this range $[0,1]$

In [ ]:
print(f"Min : {gray_image.min()}")
print(f"Max : {gray_image.max()}")

The range of colors can also be redefined by specifying the intensities corresponding to black (`vmin`) and white (`vmax`):

In [ ]:
plt.imshow(gray_image, vmin=0.0, vmax=0.6, cmap="gray");
plt.title("Intensity range = [0.0, 0.6]")
plt.colorbar();

## 2. Cropping and resizing images to standard dimensions

Resizing and cropping your images is an important first step in image preprocessing. You should crop and/or resize an image when:

* The image needs to match the input size of a machine learning model. Indeed, images come in all shapes and sizes, but machine learning algorithms typically require a standard size. You’ll want to resize and crop your images to square dimensions, often $224 \times 224$ or $256 \times 256$ pixels
* The image is too large to process efficiently. Reducing size can speed up processing
* The image needs to be displayed on a screen or webpage at a specific size

Images are all different sizes, and this is a problem for deep learning: we don’t feed the model one image at a time but several of them (what we call a mini-batch). To group them in a big array (usually called a tensor) that is going to go through our model, they all need to be of the same size. So, we need to add a transform which will resize these images to the same size.

### Image cropping

In [ ]:
#Accessing an image file from the folder
image =  io.imread('./images/Fundus_Fig1_Normal-Retina.jpg')

#Plotting the original image
plt.imshow(image)
plt.title('Color fundus photography (normal retina)\n @ Courtesy UIHC Diagnostic Imaging Staff')
plt.show()

The `skimage.util.crop` function crops an array by `crop_width` along each dimension. `crop_width` corresponds to the number of values to remove from the edges of each axis. In the following examples, `((before_top, after_bottom), … (before_left, after_right))` specifies unique crop widths at the start and end of each axis.

* **First example**

<img src="./images/crop1.png" width=500>

In [ ]:
before_top = 0
after_bottom = 100
before_left = 0
after_right = 100

cropped_image = skimage.util.crop(image, ((before_top, after_bottom), (before_left, after_right), (0,0)), copy=False)
plt.imshow(cropped_image)
plt.title('Cropped image');

print("Image dimensions before cropping:", image.shape)
print("Image dimensions after cropping:", cropped_image.shape)

* **Second example**

<img src="./images/crop2.png" width=500>

In [ ]:
before_top = 50
after_bottom = 100
before_left = 100
after_right = 100

cropped_image = skimage.util.crop(image, ((before_top, after_bottom), (before_left, after_right), (0,0)), copy=False)
plt.imshow(cropped_image)
plt.title('Cropped image');

print("Image dimensions before cropping:", image.shape)
print("Image dimensions after cropping:", cropped_image.shape)

* **Third example**

<img src="./images/crop3.png" width=500>

In [ ]:
width = image.shape[1]
height = image.shape[0]

size = 256

border_height = (height - size) // 2
border_width = (width - size) // 2
print(border_height,border_width)

cropped_image = skimage.util.crop(image, ((border_height,border_height), (border_width,border_width), (0,0)), copy=False)
plt.imshow(cropped_image)
plt.title('Cropped image');

print("Image dimensions before cropping:", image.shape)
print("Image dimensions after cropping:", cropped_image.shape)

Alternatively, cropping images can work exactly like cropping lists and arrays, by using indices to specify the range of elements to use:
    
<center>
<img src="./images/array3.png" width=400>
<img src="./images/array6.png" width=400>
</center>

In [ ]:
before_top = 50
after_bottom = 100
before_left = 100
after_right = 100

idx_top = before_top
idx_bottom = height - after_bottom
idx_left = before_left
idx_right = width - after_right
print(idx_top,idx_bottom,idx_left,idx_right)

cropped_image = image[idx_top:idx_bottom,idx_left:idx_right,:]
plt.imshow(cropped_image)

print("Image dimensions before cropping:", image.shape)
print("Image dimensions after cropping:", cropped_image.shape)

* **Fourth example**

<img src="./images/crop4.png" width=500>

To crop an image to a square, you can calculate the center square crop size use array slicing with the center coordinates. For example:

In [ ]:
width = image.shape[1]
height = image.shape[0]

size = min(width, height)

left = (width - size) // 2
top = (height - size) // 2
right = (width + size) // 2
bottom = (height + size) // 2
print(top,bottom,left,right)

cropped_image = image[top:bottom,left:right,:]
plt.imshow(cropped_image)

print("Image dimensions before cropping:", image.shape)
print("Image dimensions after cropping:", cropped_image.shape)

As you can notice, cropping images can result in losing some important details. If we crop the images then we remove some of the features that allow us to perform recognition.

### Image resizing

We can use `skimage.transform.resize()` function to resize images. This operation resizes an image by specifying an output image shape.

#### Resize the image without the same ratio

If we squish or stretch the images they end up as unrealistic shapes, leading to a model that learns that things look different to how they actually are, which we would expect to result in lower accuracy.

In [ ]:
from skimage.transform import resize

print("Image dimensions before resizing:", image.shape)
image_resized = resize(image,(256, 256))
plt.imshow(image_resized)
plt.title('Resized image')
print("Image dimensions after resizing:", image_resized.shape)

This will resize the image to $256 \times 256$ pixels.

#### Resize the image with the same aspect ratio

Most of the neural network models assume a square shape input image, which means that each image needs to be checked if it is a square or not, and cropped appropriately.

In the following example, we take the max width and length of the image and then put the image in that size.

In [ ]:
width = image.shape[1]
height = image.shape[0] 

print("Image dimensions before resizing:", height, width)

max_height = 256                                                         
max_width = 256

def adjust_size(width,height):
    
    if width > max_width or height > max_height:
        if width > height:
            return int(max_width * height/width), max_width, 3
        else:
            return max_height, int(max_height * width/height), 3
    else:
        return height, width, 3
       
image_resized = resize(image,adjust_size(width,height))

print("Image dimensions after resizing:", image_resized.shape[0], image_resized.shape[1])

plt.imshow(image_resized)
plt.title('Resized image (219x256)');

Now, you can add a center padding to the image by computing each axis offset.

In [ ]:
padding_y = (max_height - image_resized.shape[0]) / 2
padding_x = (max_width - image_resized.shape[1]) / 2

if not isinstance(padding_y, int):
    image_resized_pad = np.pad(image_resized, [(int(padding_y)+1, int(padding_y)), (int(padding_x), int(padding_x)), (0,0)]) 
elif not isinstance(padding_x, int):
    image_resized_pad = np.pad(image_resized, [(int(padding_y), int(padding_y)), (int(padding_x)+1, int(padding_x)), (0,0)]) 
else:
    image_resized_pad = np.pad(image_resized, [(int(padding_y), int(padding_y)), (int(padding_x), int(padding_x)), (0,0)]) 

print("Image dimensions after resizing:", image_resized_pad.shape)

plt.imshow(image_resized_pad)
plt.title('Resized image (256x256)');

### Image Scaling

Once you’ve ensured that all images are square (or have some predetermined aspect ratio), it’s time to scale each image appropriately. We’ve decided to have images with width and height of $128$ pixels. If the initial size was $256$, you’ll need to scale the width and height of each image by a factor of $0.5$ ($128/256$). There are a wide variety of up-scaling and down-scaling techniques and we usually use a library function to do this for us.

Rescale operation serves the same purpose as resize, but allows to specify a scaling factor instead of an output image shape instead of. The scaling factor can either be a single floating point value, or multiple values - one along each axis.

In [ ]:
from skimage.transform import rescale

image_rescaled = rescale(image_resized_pad,(0.5,0.5,1), anti_aliasing=False)

print("Image dimensions after rescaling:", image_rescaled.shape[0], image_rescaled.shape[1])

plt.imshow(image_rescaled)
plt.title('Downscaled image (128x128)')
plt.show()

## 3. Normalising pixel values for consistent brightness

When working with image data, it’s important to normalize the pixel values to have a consistent brightness and improve contrast. This makes the images more suitable for analysis and allows machine learning models to learn patterns independent of lighting conditions.

### Rescaling Pixel Values

The most common normalization technique is rescaling the pixel values to a predefined range (usually $(0,1)$ or $(-1, 1)$). This is done by dividing all pixels by the maximum pixel value (typically $255$ for RGB images). This is commonly used on different data formats, and you want to normalize all of them to apply the same algorithms over them.

Normalization is usually applied to convert an image’s pixel values to a typical or more familiar sense.

Its benefits include:

* Fairness across all images - For example, scaling all images to an equal range of $[0,1]$ or $[-1,1]$ allows all images to contribute equally to the total loss rather than when other images have high and low pixels ranges give strong and weak loss, respectively.
* Provides a standard learning rate - Since high pixel images require a low learning rate and low pixel images high learning rate, re-scaling helps provide a standard learning rate for all images.

Let’s write the code below to normalize our data. 

In [ ]:
image = io.imread('./images/skin.jpg')

In [ ]:
print(f"Min before normalization: {image.min()}")
print(f"Max before normalization: {image.max()}")
print(f"Image matrix data type before normalization: {image.dtype}")

norm_image = (image - np.min(image)) / (np.max(image) - np.min(image))

print(f"Min after normalization: {norm_image.min()}")
print(f"Max after normalization: {norm_image.max()}")
print(f"Image matrix data type after normalization: {norm_image.dtype}")

plt.imshow(norm_image);

This will scale all pixels between $0$ and $1$, with $0$ being black and $1$ being white.

## 4. Applying filters to reduce noise and sharpen images

Filtering in image processing is a very precise notion that differs from the filters proposed by some apps or softwares. In image processing, filtering consists in amplifying or attenuating some frequencies in the image. 

To properly study this notion, we should introduce two mathematical objects, namely the **Convolution** and the **Fourier transform**, thus we will consider both the usual spatial domain of the images, but also the frequency domain.  

 
Many image processing results come from a modification of one pixel with respect to its neighbors. When this modification is similar in the entire image $g$, it can be mathematically defined using a second image $h$ which defines the neighbor relationships. This results in a third image $f$. This is the so-called **convolution** and it is denoted with $*$:<br>

$$f(x,y) = (g*h)(x,y) = \sum_m \sum_n g(x-m,y-n) \ h(m,n)$$

Intuitively, the convolution “spreads” each pixel $(m,n)$ in $g$ following $h$ and proportionally to the intensity $g(m,n)$. The figure below gives an example of the computing of a particular pixel (the pixel $(2,2)$ of $f$).

<center>
<img src="./images/convolution.png" width=850>
</center>

The Fourier transform is the tool to pass from a domain to the other. To know more about the Fourier transform in signal processing, you can read this very [excellent course](https://vincmazet.github.io/signal1/fourier/fourier.html) (in French).

In the following cells, you will see how to use the function `scipy.ndimage.convolve()` to apply the convolution product.

The results of the convolutions are shown below, for both the "Dirac pulse" and the smiley.

* The function `skimage.filters.gaussian` defines a Gaussian kernel (this is the second image below). The result of the function is the convolution product of this Gaussian kernel with the image given as a parameter of the function.
* The "gradient" image (third image) brings out the vertical contours of the image.
* The two last kernels are defined with (note the double brackets, so as to get a 2D array):

```python
h = np.array([[1, -1]])
```
and
```python
N = 30
h = np.ones((1,N)) / N
```

Before convolving the image by the different filters, it is interesting to display the filters themselves. In case there is no explicit matrix to represent a filter, as with the Gaussian filter `skimage.filters.gaussian`, we can simply convolve the filter by an image containing only one non-zero pixel. Indeed, this image is equivalent to a Dirac pulse $\delta$ which is the neutral element of the convolution product.



In [ ]:
img = np.zeros((125,125))
img[62,62]=1

i, (im1, im2, im3, im4) = plt.subplots(1, 4)
i.set_figwidth(15) 

f1 = im1.imshow(img)  #Original image
plt.colorbar(f1, ax=im1, orientation='horizontal')
im1.set_title('Image')
f2 = im2.imshow(skimage.filters.gaussian(img, sigma=4))
plt.colorbar(f2, ax=im2, orientation='horizontal')
im2.set_title('Gaussian blur')
h = np.array([[1, -1]])
f3 = im3.imshow(scipy.ndimage.convolve(img,h))
plt.colorbar(f3, ax=im3, orientation='horizontal')
im3.set_title('Gradient')
N = 30
h = np.ones((1,N)) / N
f4 = im4.imshow(scipy.ndimage.convolve(img,h))
plt.colorbar(f4, ax=im4, orientation='horizontal')
im4.set_title('Motion blur')
plt.show()

In [ ]:
img = plt.imread('./images/smiley.png')

i, (im1, im2, im3, im4) = plt.subplots(1, 4)
i.set_figwidth(15) 

f1 = im1.imshow(img)  #Original image
plt.colorbar(f1, ax=im1, orientation='horizontal')
im1.set_title('Image')
f2 = im2.imshow(skimage.filters.gaussian(img, sigma=4))
plt.colorbar(f2, ax=im2, orientation='horizontal')
im2.set_title('Gaussian blur')
h = np.array([[1, -1]])
f3 = im3.imshow(scipy.ndimage.convolve(img,h))
plt.colorbar(f3, ax=im3, orientation='horizontal')
im3.set_title('Gradient')
N = 30
h = np.ones((1,N)) / N
f4 = im4.imshow(scipy.ndimage.convolve(img,h))
plt.colorbar(f4, ax=im4, orientation='horizontal')
im4.set_title('Motion blur')
plt.show()

More generally, image filters are used to reduce noise, sharpen details, and overall improve the quality of your images before analysis. Here are some of the main filters you’ll want to know about:

* **Gaussian Blur:**
The Gaussian blur filter reduces detail and noise in an image. It "blurs" the image by applying a Gaussian function to each pixel and its surrounding pixels. This can help smooth edges and details in preparation for edge detection or other processing techniques.

* **Median Blur:**
The median blur filter is useful for removing salt and pepper noise from an image. It works by replacing each pixel with the median value of its neighboring pixels. This can help smooth out isolated noisy pixels while preserving edges.

* **Laplacian Filter:**
The Laplacian filter is used to detect edges in an image. It works by detecting areas of rapid intensity change. The output will be an image with edges highlighted, which can then be used for edge detection. This helps identify and extract features in an image.

* **Unsharp Masking:**
Unsharp masking is a technique used to sharpen details and enhance edges in an image. It works by subtracting a blurred version of the image from the original image. This amplifies edges and details, making the image appear sharper. Unsharp masking can be used to sharpen details before feature extraction or object detection.

* **Bilateral Filter:**
The bilateral filter smooths images while preserving edges. It does this by considering both the spatial closeness and color similarity of pixels. Pixels that are close together spatially and similar in color are smoothed together. Pixels that are distant or very different in color are not smoothed. This results in a smoothed image with sharp edges.
The bilateral filter can be useful for noise reduction before edge detection.

By applying these filters, you’ll have high-quality, enhanced images ready for in-depth analysis and computer vision tasks. Give them a try and see how they improve your image processing results!

In [ ]:
from skimage.restoration import denoise_bilateral
from skimage import data, img_as_float
from skimage.util import random_noise

#Original MRI image
original_gray = io.imread('./images/MRI.jpg')

#Randomly adds Gaussian-distributed additive noise
sigma = 0.155
random_noisy = random_noise(original_gray, var=sigma**2)

#Replaces random pixels with either 1 or 0
sp_noisy = random_noise(original_gray, mode='s&p', amount=0.1)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(8,10), sharex=True, sharey=True)

plt.gray()

ax[0, 0].imshow(original_gray)
ax[0, 0].axis('off')
ax[0, 0].set_title('Original image')
ax[0, 1].imshow(random_noisy)
ax[0, 1].axis('off')
ax[0, 1].set_title('Gaussian noisy\n image')
ax[0, 2].imshow(sp_noisy)
ax[0, 2].axis('off')
ax[0, 2].set_title('Salt & pepper noisy\n image')

ax[1, 1].imshow(skimage.filters.gaussian(random_noisy, sigma=2))
ax[1, 1].axis('off')
ax[1, 1].set_title('Gaussian blur\n applied to gaussian noisy')
ax[1, 2].imshow(skimage.filters.median(sp_noisy))
ax[1, 2].axis('off')
ax[1, 2].set_title('Median blur\n applied to s&p noisy')
ax[1, 0].imshow(skimage.filters.unsharp_mask(original_gray))
ax[1, 0].axis('off')
ax[1, 0].set_title('Unsharp masking\n applied to original')

ax[2, 1].imshow(denoise_bilateral(random_noisy))
ax[2, 1].axis('off')
ax[2, 1].set_title('Bilateral filter\n applied to gaussian noisy')
ax[2, 2].imshow(skimage.filters.sobel(original_gray))
ax[2, 2].axis('off')
ax[2, 2].set_title('Sobel filter\n applied to original')
ax[2, 0].imshow(skimage.filters.laplace(original_gray))
ax[2, 0].axis('off')
ax[2, 0].set_title('Laplacian filter\n applied to original')

fig.tight_layout()

plt.subplots_adjust(wspace=0.1)
plt.show()

## 5. Detecting and removing background with segmentation (binarization)

Detecting and removing background from images is an important preprocessing step for many computer vision tasks. **Binarization** separates the foreground subject from the background, leaving you with a clean image containing just the subject (or sub-part of the subject).

### Thresholding

Thresholding converts a grayscale image into a binary image (black and white) by choosing a threshold value. Pixels lighter/darker than the threshold become black, and pixels darker/lighter become white. This works well for images with high contrast and uniform lighting. 

The histogram is sometimes very useful to segment the image in two classes, that is to distinguish the objects in the image with respect to their gray level. Indeed, if the histogram shows clearly two modes (i.e. two "bumps"), a threshold $T$ can be defined between these two modes, then apply a thresholding on the pixels, such that:

* if the pixel level is higher/lower that $T$, then the pixel is in class $0$
* otherwise the pixel is in class $1$

Otsu’s method automatically calculates an "optimal" threshold (marked by a red line in the histogram below) by maximizing the variance between two classes of pixels, which are separated by the threshold. Equivalently, this threshold minimizes the intra-class variance. The `skimage.filters.threshold_otsu()` method returns threshold value based on Otsu’s method. 

In [ ]:
img = io.imread('./images/TEP.png')
img = 255 - skimage.color.rgb2gray(skimage.color.rgba2rgb(img)) * 255
plt.imshow(img,cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
th = skimage.filters.threshold_otsu(img, nbins=256)
print("Threshold:", th)
th_img = (img < th) * 1

plt.subplots(2,2,figsize=(12,8))
plt.subplot(221)
plt.imshow(img,vmin=0,vmax=255,cmap='gray')
plt.title('Original image')
plt.subplot(222)
plt.imshow(th_img,cmap='Blues')
plt.imshow(img,vmin=0,vmax=255,cmap='gray',alpha=0.5)
plt.title('Overlay of original and\n binarized (in blue) images ')
plt.subplot(223)
plt.hist(img.ravel(), bins=256, range=(0,255));
plt.title('Original histogram')
plt.yticks([])
plt.axvline(th,color='red')
plt.subplot(224)
plt.hist(th_img.ravel(), bins=256, range=(0,1));
plt.yticks([])
plt.title('Binarized histogram')
plt.show()

### Region growing
Region growing starts with a group of seed points and grows outward to detect contiguous regions in an image. You provide the seed points, and the algorithm examines neighboring pixels to determine if they should be added to the region. This continues until no more pixels can be added. The `skimage.segmentation.region_growing` method implements this technique.

In [ ]:
img = io.imread('./images/TEP.png')
img = 255 - skimage.color.rgb2gray(skimage.color.rgba2rgb(img)) * 255
plt.imshow(img,cmap='gray');

In [ ]:
from skimage.morphology import flood_fill
seed1 = (150,185)
seed2 = (20,180)
new_img = flood_fill(img, seed1, 120, tolerance=80)
new_img = flood_fill(new_img, seed2, 120, tolerance=80)
plt.imshow(new_img,vmin=0,vmax=255,cmap='gray')
plt.plot(185,150, 'ro',markersize=2, label='seed 1')
plt.plot(180,20, 'bo',markersize=2, label='seed 2')
plt.legend()
plt.show()

By experimenting with these techniques, you can isolate subjects from the background in your images. Segmentation is a key first step, allowing you to focus your computer vision models on the most important part of the image-the foreground subject.

## 6. Adjusting contrast of images using histogram transformations

### Histogram Transformations

Remember, histogram represents the number of pixel with a specific intensity, for all intensities in the image. Therefore, the histogram indicates whether an image is rather dark or rather light. By applying a transformation on an histogram, one can change the pixel intensities of the image. The histogram will be used especially for segmentation. 

An histogram transformation consists in applying a mathematical function to the intensity distribution. Generally, the transformations are useful to improve the visual quality of an image, but are rarely needed inside an automatic processing.

The transform, denoted $T$, is applied on the pixel intensities to change their values: $$j=T(i)$$

where $i$ and $j$ are respectively the intensities of the new and the original image. As a consequence, the histogram of the new image differs from the histogram of the original image.

The original image and its histogram are given below. The histogram doesn't use the full range of intensities quite well. Indeed, there are few pixels with low intensities and no pixels with high intensities (above $0.6$), explaining the lack of contrast in the image.
 
Below are some common transformations (we suppose the pixel intensities to lie in $[0,1]$). Thus, what are their effect on this image?


In [ ]:
img = plt.imread('./images/chest_xray_image.png') #'./images/haze.png'
plt.subplots(1,2,figsize=(10,4))
plt.subplot(121)
plt.imshow(img,vmin=0,vmax=1,cmap='gray')
plt.title('Chest X-ray image')
plt.colorbar()
plt.subplot(122)
plt.hist(img.ravel(), bins=256, range=(0,1))
plt.title('Corresponding histogram')
plt.show()

### Histogram spreading

Histogram spreading enhances the contrast by “dilating” the histogram to the whole intensity interval.

Before the stretching can be performed it is necessary to specify the upper and lower pixel value limits over which the image is to be normalized. Often these limits will just be the minimum and maximum pixel values that the image type concerned allows. For example for 8-bit graylevel images the lower and upper limits might be $0$ and $255$. Call the lower and the upper limits $A$ and $B$ respectively.

The simplest sort of normalization then scans the image to find the lowest and highest pixel values currently present in the image. Call these $k_{max}$ and $k_{min}$. Then each pixel  is scaled using the following function:

$$T(i)=(k-k_{min})\times\frac{B-A}{k_{max}-k_{min}}+A$$

Usually, $A$ and $B$ are set to $O$ and $255$, respectively (meaning, values below $0$ are set to $0$ and values about $255$ are set to $255$).

The problem with this is that a single outlying pixel with either a very high or very low value can severely affect the value of $k_{min}$ and $k_{max}$ and this could lead to very unrepresentative scaling. Therefore a more robust approach is to first take a histogram of the image, and then select  $k_{min}$ and $k_{max}$ at, say, the 2nd and 98th percentile in the histogram (that is, $2%$ of the pixel in the histogram will have values lower than $i_{min}$, and 2% of the pixels will have values higher than $k_{max}$). The image is rescaled to include all intensities that fall within the 2nd and 98th percentiles and this prevents outliers affecting the scaling so much. 

In [ ]:
p2, p98 = np.percentile(img, (2,98))
new_img = skimage.exposure.rescale_intensity(img, in_range=(p2, p98))

plt.subplots(3,2,figsize=(10,13))
plt.subplot(321)
plt.imshow(img,vmin=0,vmax=1,cmap='gray')
plt.title('Original image')
plt.subplot(322)
plt.imshow(new_img,vmin=0,vmax=1,cmap='gray')
plt.title('Transformed image')
plt.subplot(323)
plt.hist(img.ravel(), bins=256, range=(0,1))
plt.title('Original histogram')
plt.subplot(324)
plt.hist(new_img.ravel(), bins=256, range=(0,1))
plt.title('Transformed histogram')
plt.subplot(325)
hist, bins = np.histogram(img.ravel(), bins=256, range=(0,1))
plt.plot(bins[:-1], np.cumsum(hist), 'r')
plt.title('Cumulative original histogram')
plt.subplot(326)
hist, bins = np.histogram(new_img.ravel(), bins=256, range=(0,1))
plt.plot(bins[:-1], np.cumsum(hist), 'r')
plt.title('Cumulative transformed histogram')
plt.show()

## Histogram equalization
Another useful contrast enhancing technique is histogram equalization. This spreads out the most frequent intensity values in an image and tends to make the details more visible. The equalized image has a roughly linear cumulative distribution function. 

$$T(i)=\frac{1}{MN}\sum_{k=0}^{i}{n_k}$$

where $M$ and $N$ are the image size and $n_k$ is the number of pixels with intensity $k$. This transformation aims to spread the histogram over the entire intensity range, and to make the histogram as flat as possible. A consequence is an increasing of the image contrast. It is a fully automatic method that does not require any parameters to be set. 

The histogram equalization gives the result below. The image is more contrasted than previously: this is due to the fact that the histogram of the transformed image is (almost) flat. Be careful, while histogram equalization has the advantage that it requires no parameters, it sometimes yields unnatural looking images. 

In [ ]:
from skimage import exposure
new_img = skimage.exposure.equalize_hist(img)

plt.subplots(3,2,figsize=(10,13))
plt.subplot(321)
plt.imshow(img,vmin=0,vmax=1,cmap='gray')
plt.title('Original image')
plt.subplot(322)
plt.imshow(new_img,vmin=0,vmax=1,cmap='gray')
plt.title('Transformed image')
plt.subplot(323)
plt.hist(img.ravel(), bins=256, range=(0,1))
plt.title('Original histogram')
plt.subplot(324)
plt.hist(new_img.ravel(), bins=256, range=(0,1))
plt.title('Transformed histogram')
plt.subplot(325)
hist, bins = np.histogram(img.ravel(), bins=256, range=(0,1))
plt.plot(bins[:-1], np.cumsum(hist), 'r')
plt.title('Cumulative original histogram')
plt.subplot(326)
hist, bins = np.histogram(new_img.ravel(), bins=256, range=(0,1))
plt.plot(bins[:-1], np.cumsum(hist), 'r')
plt.title('Cumulative transformed histogram')
plt.show()

In [ ]:
plt.subplots(3,3,figsize=(12,10))
plt.subplot(331)
plt.imshow(img,cmap='gray')
plt.title('Low contrast image')
plt.subplot(332)
plt.imshow(skimage.exposure.rescale_intensity(img, in_range=(p2, p98)),vmin=0,vmax=1,cmap='gray')
plt.title('Image after \nhistogram spreading')
plt.subplot(333)
plt.imshow(skimage.exposure.equalize_hist(img),vmin=0,vmax=1,cmap='gray')
plt.title('Image after \nhistogram equalization')
plt.subplot(334)
plt.hist(img.flatten(), bins=256, range=(0,1), color='C0', alpha=0.5);
plt.subplot(335)
plt.hist(skimage.exposure.rescale_intensity(img, in_range=(p2, p98)).flatten(), bins=256, range=(0,1), color='C1', alpha=0.5);
plt.subplot(336)
plt.hist(skimage.exposure.equalize_hist(img).flatten(), bins=256, range=(0,1), color='C2', alpha=0.5);
plt.subplots_adjust(wspace=0.3)
plt.subplot(337)
plt.hist(img.flatten(), cumulative=True, bins=256, range=(0,1), color='C0', alpha=0.5);
plt.subplot(338)
plt.hist(skimage.exposure.rescale_intensity(img, in_range=(p2, p98)).flatten(), cumulative=True, bins=256, range=(0,1), color='C1', alpha=0.5);
plt.subplot(339)
plt.hist(skimage.exposure.equalize_hist(img).flatten(), cumulative=True, bins=256, range=(0,1), color='C2', alpha=0.5);
plt.subplots_adjust(wspace=0.3)
#plt.subplot(3,3,(7,9))
plt.figure()
plt.hist(img.flatten(), cumulative=True, bins=256, range=(0,1), alpha=0.5, label='Low contrast image');
plt.hist(skimage.exposure.rescale_intensity(img, in_range=(p2, p98)).flatten(), cumulative=True, bins=256, range=(0,1), alpha=0.5, label='Histogram spreading');
plt.hist(skimage.exposure.equalize_hist(img).flatten(), cumulative=True, bins=256, range=(0,1), alpha=0.5, label='Histogram equalization')
plt.xlim(0,1)
plt.legend()
plt.title('Overlay of each cumulative histogram')
plt.subplots_adjust(wspace=0.25, # wspace controls the width of space between subplots
                    hspace=0.25)
plt.show()

## Using Data Augmentation to Expand Your Dataset

Data augmentation is a technique used to artificially expand the size of your dataset by generating new images from existing ones. This helps reduce overfitting and improves model accuracy, all while keeping training time and cost the same. Some common augmentation techniques for image data include:

* **Flipping and rotating:**
Simply flipping (horizontally or vertically) or rotating (90, 180, 270 degrees) images can generate new data points. For example, if you have $1,000$ images of cats, flipping and rotating them can give you $4,000$ total images ($1,000$ original + $1,000 $flipped horizontally + $1,000$ flipped vertically + $1,000$ rotated 90 degrees).

* **Cropping:**
Cropping images to different sizes and ratios creates new images from the same original. This exposes your model to different framings and compositions of the same content. You can create random crops of varying size, or target more specific crop ratios like squares.

* **Color manipulation:**
Adjusting brightness, contrast, hue, and saturation are easy ways to create new augmented images. For example, you can randomly adjust the brightness and contrast of images by up to $30%$ to generate new data points. Be careful not to distort the images too much, or you risk confusing your model.

* **Image overlays:**
Overlaying transparent images, textures or noise onto existing images is another simple augmentation technique. Adding things like watermarks, logos, dirt/scratches or Gaussian noise can create realistic variations of your original data. Start with subtle overlays and see how your model responds.

* **Combining techniques:**
For the biggest increase in data, you can combine multiple augmentation techniques on the same images. For example, you can flip, rotate, crop and adjust the color of images, generating many new data points from a single original image. But be careful not to overaugment, or you risk distorting the images beyond recognition!

Using data augmentation, you can easily multiply the size of your image dataset by $4x$, $10x$ or more, all without collecting any new images. This helps combat overfitting a

Let’s now look at the most used data augmentation techniques:

### Flipping and rotating

#### Flipping

This reverses the rows or columns of pixels in either vertical or horizontal cases, respectively.

In [ ]:
image =  io.imread('./images/epaule.jpg')

plt.subplots(2,2,figsize=(10,10))
plt.subplot(221)
plt.imshow(image[:,:])
plt.title('Original image')
plt.subplot(222)
plt.imshow(image[:,::-1])
plt.title('Flipped horizontal image')
plt.subplot(223)
plt.imshow(image[::-1,:])
plt.title('Flipped vertical image')
plt.subplot(224)
plt.imshow(image[::-1,::-1])
plt.title('Flipped both image')
plt.show()

#### Rotating

`skimage.transform` module includes tools to transform images and volumetric data.

Among them, `rotate` process involves rotating an image by a specified degree.

In [ ]:
from skimage.transform import rotate

image_rot1 = rotate(image,-30, cval=0, mode='constant') #Rotation angle in degrees in counter-clockwise direction.
image_rot2 = rotate(image,-30, mode='edge')
image_rot3 = rotate(image,-15, mode='edge', order=0)

plt.subplots(2,2,figsize=(12,10))
plt.subplot(221)
plt.imshow(image)
plt.title('Original image')
plt.subplot(222)
plt.imshow(image_rot1)
plt.title("Rotated 30° image (edge handling='constant'\ninterpolation mode='Bi-linear')")
plt.subplot(223)
plt.imshow(image_rot2)
plt.title("Rotated 30° image (edge handling='edge'\ninterpolation mode='Bi-linear')")
plt.subplot(224)
plt.imshow(image_rot3)
plt.title("Rotated 30° image\n(interpolation mode='Nearest-neighbor')")
plt.show()

### Applying any geometric transformation

Such a transform changes the shape or position of an image. It is also useful for tasks such as image registration, alignment, and geometric correction.

For example, an euclidean transformation, also known as a rigid transform has the following form:

```python
X = a0 * x - b0 * y + a1 =
  = x * cos(rotation) - y * sin(rotation) + a1

Y = b0 * x + a0 * y + b1 =
  = x * sin(rotation) + y * cos(rotation) + b1
```
where the homogeneous transformation matrix is:

```python
[[a0  a1  a2]
 [b0  b1  b2]
 [0   0    1]]
```

The Euclidean transformation is a rigid transformation with rotation and translation parameters. The similarity transformation extends the Euclidean transformation with a single scaling factor.

In 2D and 3D, the transformation parameters may be provided either via matrix, the homogeneous transformation matrix, above, or via the implicit parameters rotation and/or translation (where a1 is the translation along x, b1 along y, etc.). Beyond 3D, if the transformation is only a translation, you may use the implicit parameter translation; otherwise, you must use matrix.

In [ ]:
tform = skimage.transform.EuclideanTransform(translation=(100,50)) 
print(tform.params)

In [ ]:
tf_image = skimage.transform.warp(image, tform.inverse, mode='constant', cval=0)
plt.imshow(tf_image)
plt.title("Translated image\n (edge handling='constant')")

In [ ]:
plt.subplots(3,2,figsize=(10,15))
plt.subplot(321)
plt.imshow(image)
plt.title('Original image')
plt.subplot(322)
tform = skimage.transform.EuclideanTransform(translation=(100,50)) 
plt.imshow(skimage.transform.warp(image, tform.inverse,mode='edge'))
plt.title("Translated image (edge handling='edge')")
plt.subplot(323)
tform = skimage.transform.EuclideanTransform(translation=(100,50), rotation=np.deg2rad(15)) #Rotation angle, clockwise, as radians
plt.imshow(skimage.transform.warp(image, tform.inverse))
plt.title("Translated and rotated 15° image\n(edge handling='constant')")
plt.subplot(324)
plt.imshow(skimage.transform.warp(image, tform.inverse,mode='edge'))
plt.title("Translated and rotated 15° image\n(edge handling='edge')")
plt.subplot(325)
tform = skimage.transform.SimilarityTransform(scale = 0.7)
plt.imshow(skimage.transform.warp(image, tform.inverse,  mode='constant', cval=0))
plt.title("Scaled image (by a factor 0.7)")
plt.subplot(326)
tform = skimage.transform.SimilarityTransform(scale = 1.5)
plt.imshow(skimage.transform.warp(image, tform.inverse, mode='constant', cval=0))
plt.title("Scaled image (by a factor 1.5)")
plt.show()

As you can see these transformations work well, except that the new images are now centered in the top-left (see below). If you would like the images to remain in the original centre, this is currently possible by combining transformations.

Let's focus on scaling.

Scaling as itself is defined as one subset of affine transformations. The affine transformation matrix for scaling only is defined as

```python
[[ s_x, -0. ,  0. ],
 [ 0. ,  s_y,  0. ],
 [ 0. ,  0. ,  1. ]]
```

where `s_x` and `s_y` are the scaling factors in the respective dimensions (defined relative to the origin at $(0,0$)). If you want your image, to be scaled not relative to the origin, but another point, you first translate the image , so that the center of scaling is in the origin, then you scale, then you move the image back. You simply do a matrix multiplication of your transform matrices with the scale matrix. The result is the following


In [ ]:
shift_y, shift_x = (np.array(image.shape[:2])-1) / 2.

tf_scale = skimage.transform.SimilarityTransform(scale = 1.5) #skimage.transform.SimilarityTransform(rotation=np.deg2rad(30))
tf_shift = skimage.transform.SimilarityTransform(translation=[-shift_x, -shift_y])
tf_shift_inv = skimage.transform.SimilarityTransform(translation=[shift_x, shift_y])

image_rotated = skimage.transform.warp(image, (tf_shift + (tf_scale + tf_shift_inv)).inverse)

plt.imshow(image_rotated)
plt.title("Scaled image (by a factor 1.5)");

In [ ]:
shift_y, shift_x = (np.array(image.shape[:2])-1) / 2.
print(shift_y, shift_x)
tf_scale = skimage.transform.SimilarityTransform(scale = 0.5) #skimage.transform.SimilarityTransform(rotation=np.deg2rad(30))
tf_shift = skimage.transform.SimilarityTransform(translation=[-shift_x, -shift_y])
tf_shift_inv = skimage.transform.SimilarityTransform(translation=[shift_x, shift_y])

image_rotated = skimage.transform.warp(image, (tf_shift + (tf_scale + tf_shift_inv)).inverse, cval=0, mode='constant')

plt.imshow(image_rotated)
plt.title("Scaled image (by a factor 0.5)")

In [ ]:
shift_y, shift_x = (np.array(image.shape[:2])-1) / 2.
tf_rotate = skimage.transform.SimilarityTransform(rotation=np.deg2rad(30))
tf_shift = skimage.transform.SimilarityTransform(translation=[-shift_x, -shift_y])
tf_shift_inv = skimage.transform.SimilarityTransform(translation=[shift_x, shift_y])

image_rotated = skimage.transform.warp(image, (tf_shift + (tf_rotate + tf_shift_inv)).inverse)

plt.imshow(image_rotated)
plt.title("Rotated 30° image");

### Cropping and/or shifting

This is the process of shifting image pixels horizontally or vertically.

In [ ]:
image =  io.imread('./images/epaule.jpg')
plt.imshow(image)
plt.title('Original image');

In [ ]:
image_crop = image[:,:200]
print(image_crop.shape)
plt.imshow(image_crop)
plt.title('Cropped image');

In [ ]:
image_crop_resized = np.pad(image_crop, [(0,0), (image.shape[1]-image_crop.shape[1],0)], mode='edge')
plt.imshow(image_crop_resized)
print(image_crop_resized.shape)
plt.title("Crop padded image (left)\n (edge handling='edge')");

In [ ]:
image_crop_resized = np.pad(image_crop, [(0,0), (0,image.shape[1]-image_crop.shape[1])], mode='edge')
plt.imshow(image_crop_resized)
print(image_crop_resized.shape)
plt.title("Crop padded image (right)\n (edge handling='edge')");

In [ ]:
image_crop = image[:,100:]
print(image_crop.shape)
image_crop_resized = np.pad(image_crop, [(0,0), (image.shape[1]-image_crop.shape[1],0)], mode='edge')
plt.imshow(image_crop_resized)
print(image_crop_resized.shape)
plt.title("Crop padded image (left)\n (edge handling='edge')");

In [ ]:
image_crop = image[150:,:]
print(image_crop.shape)
image_crop_resized = np.pad(image_crop, [(0,image.shape[0]-image_crop.shape[0]), (0,0)], mode='edge')
plt.imshow(image_crop_resized)
print(image_crop_resized.shape)
plt.title("Crop padded image (bottom)\n (edge handling='edge')");

### Changing brightness

This is the process of increasing or decreasing image contrast.

In [ ]:
scan =  io.imread('./images/scan.jpeg')

plt.subplots(2,4,figsize=(15,7))
plt.subplot(241)
plt.imshow(scan,vmin=0,vmax=255)
plt.title('Original image')
plt.subplot(245)
plt.hist(scan.ravel(),bins=256, range=(0,256));
plt.ylim(0,5000)
plt.yticks([])
plt.subplot(242)
new_scan = np.clip(np.uint16(scan*0.35), 0, 255)
plt.imshow(new_scan,vmin=0,vmax=255)
plt.title('Image with lower brightness')
plt.subplot(246)
plt.hist(new_scan.ravel(),bins=256, range=(0,256));
plt.ylim(0,5000)
plt.yticks([])
plt.subplot(243)
new_scan = np.clip(np.uint16(scan*0.8), 0, 255)
plt.imshow(new_scan,vmin=0,vmax=255)
plt.title('Image with lower brightness')
plt.subplot(247)
plt.hist(new_scan.ravel(),bins=256, range=(0,256));
plt.ylim(0,5000)
plt.yticks([])
plt.subplot(244)
new_scan = np.clip(np.uint16(scan*1.5), 0, 255)
plt.imshow(new_scan,vmin=0,vmax=255)
plt.title('Image with higher brightness')
plt.subplot(248)
plt.hist(new_scan.ravel(),bins=256, range=(0,256));
plt.yticks([])
plt.ylim(0,5000)
plt.show()

# Conclusion

Having explored the popular and commonly used image preprocessing techniques, what now remains is modeling your machine learning models to the desired level of high accuracy and performance. Thus, we are now ready to jump into building custom computer vision projects.

Good luck!